在工作中，我们经常遇到将Excel表中的部分信息填写到Word文档的对应表格中，以生成报告，方便打印。今天我们就来试试如何往固定的Word模板中填写数据。此例要求将Excel档的“收货数据”填写到Word档的“收货记录”中去，并加总收货数量。模板是下面这样的。
![](images\temp.png)
要求结果是这样的：
![](images\result.png)

首先，需要从Excel表格中提取数据。

In [1]:
#从Excel表中提取数据
from openpyxl import load_workbook
wb = load_workbook("收货数据.xlsx")
ws = wb.active
data=[]
for row in range(2,ws.max_row+1):
    seq = ws["A"+str(row)].value
    supplier = ws["B"+str(row)].value
    material_pn = ws["C"+str(row)].value
    material_model = ws["D"+str(row)].value
    desp = ws["E"+str(row)].value
    qty = ws["F"+str(row)].value
    date = ws["G"+str(row)].value.date()
    info = [seq, supplier, material_pn, material_model, desp, qty, date]
    data.append(info)

In [2]:
#检查一下数据
data[0]

[1,
 '供应商HH',
 'PROYY-0001',
 'PROYY-0001CP*-SM00001',
 'SM00001',
 165,
 datetime.date(2019, 1, 1)]

由于需要将数量列的数据加总，所以先定义一个加总函数，以便后面调用。

In [3]:
#定义数量加总函数
def Sum_list(list):
    s = 0
    for i in list: #累加列表中的所有数
        s+=i
    return s

In [4]:
#加总数量列的所有数字
qty_list=[]
for i in data:
    qty_list.append(i[5]) #数量在内层列表的第6个位置，索引是5
sum_qty= Sum_list(qty_list) #调用加总函数加总
sum_qty

2307

定义单元格合并函数，实例28有详细解释，可移步https://mp.weixin.qq.com/s/d4JTViIZA5GyDmwKlow8Ug 了解。

In [5]:
#定义合并单元格的函数    
def Merge_cells(table,target_list,start_row,col):
    '''
    table: 是需要操作的表格
    target_list: 是目标列表，即含有重复数据的列表
    start_row: 是开始行，即表格中开始比对数据的行（需要将标题除开）
    col: 是需要处理数据的列
    '''
    start = 0 #开始行计数
    end = 0 #结束行计数
    reference = target_list[0] #设定基准，以列表中的第一个字符串开始
    for i in range(len(target_list)): #遍历列表
        if target_list[i] != reference: #开始比对，如果内容不同执行如下
            reference = target_list[i] #基准变成列表中下一个字符串
            end = i - 1 
            table.cell(start+start_row,col).merge(table.cell(end+start_row,col))
            start = end + 1
        if i == len(target_list) - 1: #遍历到最后一行，按如下操作
            end = i
            table.cell(start+start_row,col).merge(table.cell(end+start_row,col))

然后往word的表格中写入数据，并合并单元格。

In [6]:
#数据提取即处理完毕后，就可以往Word的表格中写入数据了
from docx import Document
doc = Document("收货记录模板.docx")
#读取word文档中的第一个表格的第二和第三列除标题和尾部总数行的数据
table = doc.tables[0] #已确定是第一个表格，其索引是0
supplier = [] #存储供应商名称
pn = [] #存储物料编码
for i in data:
    supplier.append(i[1])
    pn.append(i[2])
#按需增加行，以便填写数据
for i in range(len(supplier)): #模板中已经有一行了，所以总共只需增加len(supplier)行
    table.add_row() 
#增加好行后先做合并单元格操作
Merge_cells(table,supplier,1,1) #开始合并行为2，索引为1；供应商名称是在2列，索引为1
Merge_cells(table,pn,1,2) #开始合并行为2，索引为1；物料编码是在3列，索引为2

#写入数据到表格
for row in range(1,len(supplier)+1):
    for col in range(7):        
        table.cell(row,col).text = str(data[row-1][col])

max_row = len(table.rows) #获取最大一行
qty_row = max_row-1 #确定需要写入加总数据的一行
table.cell(qty_row,5).merge(table.cell(qty_row,5)) #合并右下角用于填写数量的两个单元格
table.cell(qty_row,4).text = '总数：'
table.cell(qty_row,5).text = str(sum_qty)

doc.save("收货记录.docx")

由于合并供应商名称和物料编码两列中的相同内容的单元格，所有需要把这两列的信息从总数据`data`中单独提取出来。新建了`supplier`和`pn`两个空列表，然后遍历`data`，将数据取出并存入空列表。随后，我们根据实际需要，新增一些空白行，以便后面写入数据。新增行的数量以`supplier`列表中元素的个数为依据。这个表格已经存在一个空白行，算上外行要填“总数”，只需增加`supplier`列表中元素的个数对应那么多行就够了。当然这里也可以以列表`pn`中元素个数为依据，两者是相等的。增加好行后，先调用函数`Merge_cells`做合并单元格操作（如果先写入数据，再合并，会导致单元格内容重复）。合并完后，就可以写入数据到表格了，此处遍历需要写入数据的行和列，然后从`data`中提取数据。数据行写完后，还需要写入最后一行的“总数”字样和总的物料数量。最后保存数据，得到如下结果：
![](images\result-actual.png)

看起来是不是有点丑啊，那还需要调整一下字体加粗，单元格对齐，单元格框线，行高参数。这些留到下一个实例再做吧，记得“追剧”哦^_^。